# Assignment: Skill Chaining and Latency Monitoring in Semantic Kernel


---

### Objective:
This assignment focuses on understanding and implementing **skill chaining** within the **Semantic Kernel** framework. You will design and connect multiple AI functions (skills) to create a complex workflow. Crucially, you will also **monitor the latency** of these chained operations to analyze the performance implications of sequential AI calls.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM service supported by Semantic Kernel (e.g., OpenAI, Azure OpenAI). Configure your API key(s) securely.
2.  **Semantic Kernel Setup**: Install the `semantic-kernel` library (`pip install semantic-kernel`).
3.  **Scenario: Content Creation Pipeline**: Simulate a content creation pipeline where text is processed through several stages:
    * **Skill 1: Summarization**: Takes a long article and produces a concise summary.
    * **Skill 2: Tone Adjustment**: Takes the summary and rewrites it to a specific tone (e.g., formal to informal, neutral to enthusiastic).
    * **Skill 3: Keyword Extraction**: Extracts relevant keywords from the adjusted summary.
    * **Skill 4 (Optional/Advanced): Title Generation**: Generates catchy titles based on the adjusted summary and keywords.
4.  **Skill Chaining**: Implement this pipeline by defining each step as a distinct Semantic Kernel skill (Semantic Functions). You must demonstrate how the output of one skill serves as the input for the next, forming a chain.
5.  **Latency Monitoring**: Integrate a mechanism to measure the execution time of each individual skill and the total time taken for the entire chained operation. Use Python's `time` module or similar.
6.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
7.  **Analysis**: Explain your skill design, chaining implementation, and the observed latency measurements. Discuss the factors influencing latency and potential optimization strategies.

---

## Part 1: Setup and Semantic Kernel Configuration
Begin by setting up your environment and configuring Semantic Kernel with your LLM.

### Task 1.1: Install Libraries and Configure Semantic Kernel
Install `semantic-kernel` and `python-dotenv`. Set up your LLM API key. We'll use OpenAI for this example.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install semantic-kernel python-dotenv --quiet

import semantic_kernel as sk
import os
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- IMPORTANT: Create a .env file in the same directory as this notebook with the line: ---
# OPENAI_API_KEY="YOUR_OPENAI_API_KEY_HERE"
# OPENAI_CHAT_MODEL="gpt-3.5-turbo" # or "gpt-4o", "gpt-4"

# Initialize the kernel
kernel = sk.Kernel()

# Configure LLM (OpenAI example)
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL", "gpt-3.5-turbo")

if not api_key:
    print("WARNING: OPENAI_API_KEY not found in environment variables. Please set it in .env file.")
else:
    kernel.add_service(
        sk.OpenAIChatCompletion(
            service_id="chat_completion",
            ai_model_id=model_id,
            api_key=api_key
        ),
    )
    print(f"Semantic Kernel initialized with OpenAI model: {model_id}")

# Example long article for processing
long_article = (
    "The recent surge in AI advancements has led to significant discussions about its societal impact. "
    "From automated customer service to complex scientific research, artificial intelligence is reshaping industries "
    "at an unprecedented pace. However, alongside the excitement, concerns about job displacement, "
    "ethical implications, and data privacy have also gained prominence. Governments and organizations "
    "worldwide are grappling with the challenge of regulating AI development to ensure responsible innovation. "
    "Bias in algorithms, explainability of AI decisions, and the potential for misuse are critical areas "
    "requiring careful consideration. Public discourse is crucial to fostering a balanced understanding "
    "of AI's potential benefits and risks. Many experts believe that collaboration between technologists, "
    "policymakers, and ethicists is vital for navigating this transformative era successfully. "
    "Investment in AI education and retraining programs is also seen as essential to prepare the workforce "
    "for the future. Ultimately, the trajectory of AI's integration into society will depend on a collective "
    "commitment to ethical guidelines and human-centric design."
)

print("Sample article defined.")

---

## Part 2: Define Semantic Kernel Skills
Create separate Semantic Functions for each step of the content creation pipeline.

### Task 2.1: Summarization Skill
Create a skill that takes an input text and summarizes it.

In [ ]:
summarize_skill_prompt = """
{{$input}}

Given the text above, provide a concise summary of 2-3 sentences. Focus on the main points.
"""

summarize_function = kernel.create_function_from_prompt(
    function_name="Summarize",
    plugin_name="ContentProcessing",
    prompt=summarize_skill_prompt,
    prompt_template_config=sk.PromptTemplateConfig(
        input_variables=[sk.InputVariable(name="input", description="The text to summarize.")],
        execution_settings=[sk.OpenAIChatCompletion.DEFAULT_MAX_TOKENS]
    )
)

print("Summarization skill created.")

### Task 2.2: Tone Adjustment Skill
Create a skill that takes an input text (the summary) and adjusts its tone.

In [ ]:
tone_adjust_skill_prompt = """
Rewrite the following text in a more {{$tone}} tone:

{{$input}}
"""

tone_adjust_function = kernel.create_function_from_prompt(
    function_name="AdjustTone",
    plugin_name="ContentProcessing",
    prompt=tone_adjust_skill_prompt,
    prompt_template_config=sk.PromptTemplateConfig(
        input_variables=[
            sk.InputVariable(name="input", description="The text to adjust tone."),
            sk.InputVariable(name="tone", description="The desired tone (e.g., 'enthusiastic', 'formal', 'casual').")
        ],
        execution_settings=[sk.OpenAIChatCompletion.DEFAULT_MAX_TOKENS]
    )
)

print("Tone Adjustment skill created.")

### Task 2.3: Keyword Extraction Skill
Create a skill that takes an input text (the adjusted summary) and extracts relevant keywords.

In [ ]:
keyword_extract_skill_prompt = """
Extract 3-5 key terms or phrases from the following text, separated by commas:

{{$input}}
"""

keyword_extract_function = kernel.create_function_from_prompt(
    function_name="ExtractKeywords",
    plugin_name="ContentProcessing",
    prompt=keyword_extract_skill_prompt,
    prompt_template_config=sk.PromptTemplateConfig(
        input_variables=[sk.InputVariable(name="input", description="The text to extract keywords from.")],
        execution_settings=[sk.OpenAIChatCompletion.DEFAULT_MAX_TOKENS]
    )
)

print("Keyword Extraction skill created.")

### Task 2.4 (Optional/Advanced): Title Generation Skill
Create a skill that generates catchy titles based on the summary and keywords.

In [ ]:
title_generate_skill_prompt = """
Based on the following summary and keywords, suggest 3 catchy and relevant titles, each on a new line.

Summary: {{$summary}}
Keywords: {{$keywords}}
"""

title_generate_function = kernel.create_function_from_prompt(
    function_name="GenerateTitles",
    plugin_name="ContentProcessing",
    prompt=title_generate_skill_prompt,
    prompt_template_config=sk.PromptTemplateConfig(
        input_variables=[
            sk.InputVariable(name="summary", description="The content summary."),
            sk.InputVariable(name="keywords", description="Extracted keywords.")
        ],
        execution_settings=[sk.OpenAIChatCompletion.DEFAULT_MAX_TOKENS]
    )
)

print("Title Generation skill created (optional).")

---

## Part 3: Skill Chaining and Latency Monitoring
Implement the sequential execution of your skills and measure the time taken for each step and the overall pipeline.

### Task 3.1: Execute Chained Skills with Latency Measurement
Write asynchronous Python code to execute the skills in sequence, passing the output of one as input to the next. Measure and print the latency for each step and the total latency.

In [ ]:
async def run_content_pipeline(article_text: str, desired_tone: str):
    print(f"\n--- Starting Content Pipeline for Tone: '{desired_tone}' ---")
    total_pipeline_start_time = time.time()
    latency_measurements = {}

    # Step 1: Summarization
    step_start_time = time.time()
    summary_result = await kernel.invoke(summarize_function, input=article_text)
    summary_output = str(summary_result)
    latency_measurements["Summarization"] = time.time() - step_start_time
    print(f"Summary ({latency_measurements['Summarization']:.2f}s): {summary_output}")

    # Step 2: Tone Adjustment
    step_start_time = time.time()
    tone_adjusted_result = await kernel.invoke(tone_adjust_function, input=summary_output, tone=desired_tone)
    tone_adjusted_output = str(tone_adjusted_result)
    latency_measurements["Tone Adjustment"] = time.time() - step_start_time
    print(f"Tone Adjusted ({latency_measurements['Tone Adjustment']:.2f}s): {tone_adjusted_output}")

    # Step 3: Keyword Extraction
    step_start_time = time.time()
    keywords_result = await kernel.invoke(keyword_extract_function, input=tone_adjusted_output)
    keywords_output = str(keywords_result)
    latency_measurements["Keyword Extraction"] = time.time() - step_start_time
    print(f"Keywords ({latency_measurements['Keyword Extraction']:.2f}s): {keywords_output}")

    # Step 4 (Optional): Title Generation
    titles_output = "N/A"
    if title_generate_function: # Check if the optional skill was created
        step_start_time = time.time()
        titles_result = await kernel.invoke(title_generate_function, summary=tone_adjusted_output, keywords=keywords_output)
        titles_output = str(titles_result)
        latency_measurements["Title Generation"] = time.time() - step_start_time
        print(f"Titles ({latency_measurements['Title Generation']:.2f}s):\n{titles_output}")

    total_pipeline_end_time = time.time()
    total_latency = total_pipeline_end_time - total_pipeline_start_time
    latency_measurements["Total Pipeline"] = total_latency

    print("\n--- Pipeline Summary ---")
    print("Individual Skill Latencies:")
    for skill, latency in latency_measurements.items():
        if skill != "Total Pipeline":
            print(f"- {skill}: {latency:.2f} seconds")
    print(f"Total Pipeline Latency: {total_latency:.2f} seconds")
    print("------------------------")

    return {
        "summary": summary_output,
        "adjusted_content": tone_adjusted_output,
        "keywords": keywords_output,
        "titles": titles_output,
        "latency": latency_measurements
    }

print("run_content_pipeline function defined.")

### Task 3.2: Run the Pipeline
Execute the pipeline with your sample article and observe the outputs and latencies.

In [ ]:
import asyncio

# Run the pipeline with an 'enthusiastic' tone
results_enthusiastic = await run_content_pipeline(long_article, "enthusiastic")

# You can run it again with a different tone to see variations
print("\n" + "="*50 + "\n")
results_formal = await run_content_pipeline(long_article, "formal")

---

## Part 4: Analysis and Reflection
Based on your implementation and observations, answer the following questions.

### Task 4.1: Skill Chaining Effectiveness
* **Design**: How effectively did you design your skills to enable chaining? Describe how the output of one skill became the input for the next.
* **Flexibility**: How does this chaining approach contribute to the modularity and reusability of your AI functions?
* **Prompt Engineering**: Discuss any challenges faced in prompt engineering to ensure smooth transitions and consistent output quality across chained skills.

### Task 4.2: Latency Analysis
* **Observations**: Report the individual skill latencies and the total pipeline latency you observed. Are there significant differences between skill latencies? What might explain these differences?
* **Factors Influencing Latency**: Beyond the number of skills, what other factors do you think contributed to the overall latency (e.g., LLM model choice, complexity of prompt, token count, API network latency, LLM provider's load)?
* **Impact of Chaining**: How does chaining multiple LLM calls sequentially impact the overall user experience, especially compared to a single, more complex LLM call (if it were possible)?
* **Optimization Strategies**: Propose at least two strategies to reduce latency in a real-world skill-chained application (e.g., parallelization, caching, smaller models, prompt optimization, batched processing).

### Task 4.3: Practical Applications and Robustness
* **Real-world Use Cases**: Provide examples of real-world scenarios where skill chaining with latency monitoring would be crucial (e.g., intelligent chatbots, automated report generation, data processing workflows).
* **Error Handling**: How would you enhance this pipeline to handle errors gracefully (e.g., if one skill fails due to an LLM error or returns unexpected output)?
* **Monitoring beyond Latency**: What other metrics, besides latency, would be important to monitor for a production-ready skill-chained system (e.g., token usage, cost, output quality, error rates)?

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Make sure your `.env` file (or equivalent API key setup) is mentioned but **NOT** included in the submitted notebook for security reasons.
* Save your Jupyter Notebook as `[YourName]_SemanticKernel_Chaining_Latency_Assignment.ipynb`.